In [1]:
#ignore warnings
import warnings
warnings.filterwarnings("ignore")

#wrangling
import pandas as pd
import numpy as np

#explore
import scipy.stats as stats

#visuals
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from env import user, password, host

#default pandas decimal display formatting
pd.options.display.float_format='{:20,.2f}'.format

# Acquire

1.) Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

Be sure to do the correct join. We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid
only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property id's), along with zestimate error and date of transaction.
only include properties that include a latitude and longitude value

In [2]:
url = f'mysql+pymysql://{user}:{password}@{host}/zillow'

In [3]:
zillow = pd.read_sql("""select p_17.parcelid, logerror, transactiondate, p.*
    from predictions_2017 p_17
    join 
        (select
        parcelid, Max(transactiondate) as tdate
        from predictions_2017
        group By parcelid )as sq1
    on (sq1.parcelid=p_17.parcelid and sq1.tdate = p_17.transactiondate )
    join properties_2017 p on p_17.parcelid=p.parcelid
    where (p.latitude is not null and p.longitude is not null)""",url)

2.) Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [4]:
zillow.head()

,parcelid,logerror,transactiondate,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,14297519,0.03,2017-01-01,1727539,14297519,nan,nan,nan,3.50,4.00,...,nan,nan,"485,713.00","1,023,282.00","2,016.00","537,569.00","11,013.72",None,nan,"60,590,630,072,012.00"
1,17052889,0.06,2017-01-01,1387261,17052889,nan,nan,nan,1.00,2.00,...,1.00,nan,"88,000.00","464,000.00","2,016.00","376,000.00","5,672.48",None,nan,"61,110,010,023,006.00"
2,14186244,0.01,2017-01-01,11677,14186244,nan,nan,nan,2.00,3.00,...,1.00,nan,"85,289.00","564,778.00","2,016.00","479,489.00","6,488.30",None,nan,"60,590,218,022,012.00"
3,12177905,-0.10,2017-01-01,2288172,12177905,nan,nan,nan,3.00,4.00,...,nan,nan,"108,918.00","145,143.00","2,016.00","36,225.00","1,777.51",None,nan,"60,373,001,001,006.00"
4,10887214,0.01,2017-01-01,1970746,10887214,1.00,nan,nan,3.00,3.00,...,nan,nan,"73,681.00","119,407.00","2,016.00","45,726.00","1,533.89",None,nan,"60,371,236,012,000.00"


In [5]:
zillow.shape

(77381, 62)

zillow.info

3.) Write a function that takes in a dataframe of observations and attributes and returns a df where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [6]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    pct_missing = num_missing/rows
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'pct_rows_missing': pct_missing})
    return cols_missing

In [7]:
nulls_by_col(zillow)

,num_rows_missing,pct_rows_missing
parcelid,0,0.00
logerror,0,0.00
transactiondate,0,0.00
id,0,0.00
parcelid,0,0.00
airconditioningtypeid,52428,0.68
architecturalstyletypeid,77175,1.00
basementsqft,77331,1.00
bathroomcnt,0,0.00
bedroomcnt,0,0.00


In [8]:
zillow.shape

(77381, 62)

4.) Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [9]:
def nulls_by_row(df):
    num_cols_missing = df.isnull().sum(axis=1)
    pct_cols_missing = df.isnull().sum(axis=1)/df.shape[1]*100
    rows_missing = pd.DataFrame({'num_cols_missing': num_cols_missing, 'pct_cols_missing': pct_cols_missing}).reset_index().groupby(['num_cols_missing','pct_cols_missing']).count().rename(index=str, columns={'index': 'num_rows'}).reset_index()
    return rows_missing

In [10]:
nulls_by_row(zillow)

,num_cols_missing,pct_cols_missing,num_rows
0,19,30.64516129032258,2
1,20,32.25806451612903,10
2,21,33.87096774193548,83
3,22,35.483870967741936,262
4,23,37.096774193548384,599
5,24,38.70967741935484,660
6,25,40.32258064516129,5768
7,26,41.935483870967744,7996
8,27,43.54838709677419,12410
9,28,45.16129032258064,20660


In [11]:
zillow.head()

,parcelid,logerror,transactiondate,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,14297519,0.03,2017-01-01,1727539,14297519,nan,nan,nan,3.50,4.00,...,nan,nan,"485,713.00","1,023,282.00","2,016.00","537,569.00","11,013.72",None,nan,"60,590,630,072,012.00"
1,17052889,0.06,2017-01-01,1387261,17052889,nan,nan,nan,1.00,2.00,...,1.00,nan,"88,000.00","464,000.00","2,016.00","376,000.00","5,672.48",None,nan,"61,110,010,023,006.00"
2,14186244,0.01,2017-01-01,11677,14186244,nan,nan,nan,2.00,3.00,...,1.00,nan,"85,289.00","564,778.00","2,016.00","479,489.00","6,488.30",None,nan,"60,590,218,022,012.00"
3,12177905,-0.10,2017-01-01,2288172,12177905,nan,nan,nan,3.00,4.00,...,nan,nan,"108,918.00","145,143.00","2,016.00","36,225.00","1,777.51",None,nan,"60,373,001,001,006.00"
4,10887214,0.01,2017-01-01,1970746,10887214,1.00,nan,nan,3.00,3.00,...,nan,nan,"73,681.00","119,407.00","2,016.00","45,726.00","1,533.89",None,nan,"60,371,236,012,000.00"


# Prepare

1.) Remove any properties that are likely to be something other than a single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [12]:
zillow = pd.read_sql("""select *
    from predictions_2017 p_17
    join 
        (select
        parcelid, Max(transactiondate) as tdate
        from predictions_2017
        group By parcelid )as sq1
    on (sq1.parcelid=p_17.parcelid and sq1.tdate = p_17.transactiondate )
    join properties_2017 p on p_17.parcelid=p.parcelid
    left outer join airconditioningtype using (airconditioningtypeid)
    left outer join architecturalstyletype using (architecturalstyletypeid)
    left outer join buildingclasstype using (buildingclasstypeid)
    left outer join heatingorsystemtype using (heatingorsystemtypeid)
    left outer join propertylandusetype using (propertylandusetypeid)
    left outer join storytype using (storytypeid)
    left outer join typeconstructiontype using (typeconstructiontypeid)
    where (p.latitude is not null and p.longitude is not null)
    and transactiondate >= '2017-01-01'
    and transactiondate <= '2017-12-31'
    and bedroomcnt > 0 
    and bathroomcnt > 0
    and calculatedfinishedsquarefeet >= 400
    and propertylandusetypeid not in ('31', '290', '291', '246', '247', '248', '263', '264', '265', '266','267', '270', '271', '275');""",url)

In [13]:
zillow.shape

(54099, 72)

In [14]:
zillow.head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,parcelid,logerror,...,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,nan,nan,261.00,nan,None,nan,nan,0,14297519,0.03,...,None,nan,"60,590,630,072,012.00",None,None,None,None,Single Family Residential,None,None
1,nan,nan,261.00,nan,None,nan,nan,1,17052889,0.06,...,None,nan,"61,110,010,023,006.00",None,None,None,None,Single Family Residential,None,None
2,nan,nan,261.00,nan,None,nan,nan,2,14186244,0.01,...,None,nan,"60,590,218,022,012.00",None,None,None,None,Single Family Residential,None,None
3,nan,nan,261.00,2.00,None,nan,nan,3,12177905,-0.10,...,None,nan,"60,373,001,001,006.00",None,None,None,Central,Single Family Residential,None,None
4,nan,nan,261.00,2.00,None,nan,1.00,6,12095076,-0.00,...,None,nan,"60,374,608,001,014.00",Central,None,None,Central,Single Family Residential,None,None


2.) Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

The input:
- a dataframe
- a number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing)
- a number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. i.e. if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing)

The output:
- the dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.

Hint:
Look up the dropna documentation
you will want to compute a threshold from your input values (prop_required) and total number of rows or columns. 
Make use of inplace

In [15]:
zillow.shape

(54099, 72)

In [16]:
def handle_missing_values(df, prop_required_column, prop_required_row):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [17]:
handle_missing_values(zillow, .8, .8)
zillow.shape

(54099, 33)

3.) Decide how to handle the remaining missing values

- fill with constant value
- impute with mean, median, mode
- drop row/column

In [18]:
nulls_by_col(zillow)

,num_rows_missing,pct_rows_missing
propertylandusetypeid,0,0.00
id,0,0.00
parcelid,0,0.00
logerror,0,0.00
transactiondate,0,0.00
parcelid,0,0.00
tdate,0,0.00
id,0,0.00
parcelid,0,0.00
bathroomcnt,0,0.00


In [19]:
nulls_by_row(zillow)

,num_cols_missing,pct_cols_missing,num_rows
0,0,0.0,52298
1,1,3.0303030303030303,1674
2,2,6.0606060606060606,72
3,3,9.090909090909092,36
4,4,12.121212121212121,17
5,5,15.151515151515152,2
